In [11]:
import numpy as np
import shutil

"""
Creating some data links that mimic the folder and filename structure expected after MLCommons data prep
Taking BraTS22 data and dumping it into a files matching that of data prep results.
Will be used to shard into multiple insitutions, like Evan wants to in oreder to simulate NNUnet FL over sythetic insitutions

"""
import os


num_samples_one_col = 4
timestamps_random = True
timestamp_limit = 2    # if not timestamps_random. will use this value
test_name = 'random_times'
allow_overwrite = True



source_dirpath = '/raid/datasets/BraTS22/BraTS2022_Training/RSNA_ASNR_MICCAI_BraTS2021_TrainingData_16July2021'
test_folderpaths = [f'/raid/edwardsb/projects/RANO/test_data_links_{test_name}_0', f'/raid/edwardsb/projects/RANO/test_data_links_{test_name}_1']
if allow_overwrite:
    for path in test_folderpaths:
        shutil.rmtree(path)
sample_indices = [(0, num_samples_one_col), (num_samples_one_col, 2*num_samples_one_col)]

src_mods = ['t1.nii.gz', 't2.nii.gz', 't1ce.nii.gz', 'flair.nii.gz', 'seg.nii.gz']
dst_mods = ['t1n.nii.gz', 't2w.nii.gz', 't1c.nii.gz', 't2f.nii.gz', 'final_seg.nii.gz']

patient_to_timestamps = {}

for test_folderpath, indices in zip(test_folderpaths, sample_indices):
    patients = list(os.listdir(source_dirpath))[indices[0]:indices[1]]
    for src_mod, dst_mod in zip(src_mods, dst_mods):
        if src_mod == 'seg.nii.gz':
            subdir = 'labels'
        else:
            subdir = 'data'
        
        for patient in patients:
            if src_mod == src_mods[0]:
                if timestamps_random:
                    patient_to_timestamps[patient] = ['2008.03.26', '2008.12.11', '2008.12.20'][:np.random.randint(1,3)]
                else:
                    patient_to_timestamps[patient] = ['2008.03.26', '2008.12.11', '2008.12.20'][:timestamp_limit]

            dst_pat_dir = os.path.join(test_folderpath, subdir, patient)
            if not os.path.exists(dst_pat_dir):
                os.makedirs(dst_pat_dir)
            src_pat_dir = os.path.join(source_dirpath, patient)
            for timestamp in patient_to_timestamps[patient]:
                dst_pattim_dir = os.path.join(dst_pat_dir, timestamp)
                if not os.path.exists(dst_pattim_dir):
                    os.makedirs(dst_pattim_dir)
                src_path = os.path.join(src_pat_dir, patient + '_' + src_mod)
                if dst_mod == 'final_seg.nii.gz':
                    dst_path = os.path.join(dst_pattim_dir, patient + '_' + timestamp + '_' + dst_mod)
                else:
                    dst_path = os.path.join(dst_pattim_dir, patient + '_' + timestamp + '_brain_' + dst_mod)
                os.symlink(src=src_path, dst=dst_path)

print(test_folderpaths)

['/raid/edwardsb/projects/RANO/test_data_links_random_times_0', '/raid/edwardsb/projects/RANO/test_data_links_random_times_1']
